In [139]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from datetime import date
import pandas as pd
import numpy as np 
import plotly.figure_factory as ff

from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns
import random 
import warnings
import operator
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

Possible improvement:
    - more FE with geographical averages
    - Row imputation with PCA istead of filling with 0 (for 31 values is meaneduc)
    

In [2]:
#need to figure out a way to manage missing values

## Exploration of the data

In [173]:

train = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/train.csv", nrows = None)
test = pd.read_csv("../Costa_rica_poverty_prediction/costa-rican-household-poverty-prediction/test.csv")


#how to set wdir ??
print ("Train Dataset: Rows, Columns: ", train.shape)
print ("Test Dataset: Rows, Columns: ", test.shape)
#weird that we have bigger test set than train set

Train Dataset: Rows, Columns:  (9557, 143)
Test Dataset: Rows, Columns:  (23856, 142)


## Feature engineering

Code written by me

In [149]:
def feature_creation_function(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('bed_density', 'bedrooms', 'rooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('person_per_room', 'r4t3', 'rooms'),
                 ('person_per_room', 'r4t3', 'bedrooms'),
                 ('rez_esc_escolari', 'rez_esc', 'escolari'),
                 ('rez_esc_r4t1', 'rez_esc', 'r4t1'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 ('male_over_female', 'r4h3', 'r4m3'),
                 ('man12plus_over_women12plus', 'r4h2', 'r4m2'),
                 ('pesioner_over_working', 'hogar_mayor', 'hogar_adul'),
                 ('children_over_working', 'hogar_nin', 'hogar_adul'),
                 ('education_fraction', 'escolari', 'age')
                 #('', '', ''),
                ]
    
    
  
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('non_bedrooms', 'rooms', 'bedrooms'),
                 ('people_not_linving_', 'tamhog', 'r4t3')
                 ]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
        
    df['edjef'] = np.max(df[['edjefa','edjefe']], axis=1).astype(np.float32) #creating featur for max of either head of households
    


    
    

In [150]:
#uses do_features

def preprocess(df): #reencode idhogar
    # encode the idhogar - houshold level identifier
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])
    # create aggregate features
    

  

In [151]:
#mapping yes/no to float values
def clean_column(df):
    df['dependency'] = df['dependency'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefe'] = df['edjefe'].replace({"yes": 1, "no": 0}).astype(np.float64)
    df['edjefa'] = df['edjefa'].replace({"yes": 1, "no": 0}).astype(np.float64)
    #Some rows indicate that household have and don't have toilets - we fix the inconsistency by setting 0 if they have no water
    df.loc[(df.v14a ==  1) & (df.sanitario1 ==  1) & (df.abastaguano == 0), "v14a"] = 0
    df.loc[(df.v14a ==  1) & (df.sanitario1 ==  1) & (df.abastaguano == 0), "sanitario1"] = 0
    #'v18q1' only missing when 'v18q' is 0 So will fill with 0 
    df['v18q1']=df['v18q1'].fillna(0)
    # As discussed in https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403#, missing values or rent payment can be interpreted as 0 payment
    df['v2a1']=df['v2a1'].fillna(0)
    #We are missing most of missing years in school, it seems like this column was not filled in much in the survey, we will drop the column
    df['rez_esc']=df['rez_esc'].fillna(0)
    #Same goes here, we have only 31 missing values for the whole test set and 0 for the train set. We just impute them with 0 rther than using a more complicated methods for now
    df['meaneduc']=df['meaneduc'].fillna(0)
    df['SQBmeaned']=df['SQBmeaned'].fillna(0)
    #df.loc[test.meaneduc.isnull(), "meaneduc"] = 0
    #df.loc[test.SQBmeaned.isnull(), "SQBmeaned"] = 0


    


In [152]:
def recoding_ordinal(df):  #create ordinal variable instead of OHE   
    df['walls'] = np.argmax(np.array(df[['epared1', 'epared2', 'epared3']]), axis = 1) #Get quality indexes
    df['roof'] = np.argmax(np.array(df[['etecho1', 'etecho2', 'etecho3']]), axis = 1)
    df['floor'] = np.argmax(np.array(df[['eviv1', 'eviv2', 'eviv3']]), axis = 1)
    df['house_qual']= df['walls'] + df['roof'] + df['floor']
    
    power =[]
    for i, row in df.iterrows():
        if row['noelec'] == 1:
            power.append(0)
        elif row['coopele'] == 1:
            power.append(1)
        elif row['public'] == 1:
            power.append(2)
        elif row['planpri'] == 1:
            power.append(3)
        else:
            power.append(np.nan)
            
    df['power'] = power
    df['power-missing'] = df['power'].isnull()  
    df['danger'] = 1 * (df['sanitario1'] + (df['power'] == 0) + df['pisonotiene'] + df['abastaguano'] + (df['cielorazo'] == 0))     


In [153]:
def agg_function_household(df): #create a lot of features
    
    list_aggr_mean = ['rez_esc', 'dis', 'male', 'female', 'estadocivil1', 'estadocivil2', 'estadocivil3', 'estadocivil4', 'estadocivil5', 'estadocivil6', 'estadocivil7', 'parentesco2',
             'parentesco3', 'parentesco4', 'parentesco5', 'parentesco6', 'parentesco7', 'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11', 'parentesco12',
             'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',]

    list_other = ['escolari', 'age', 'fe_education_fraction']

    df_2 = pd.DataFrame()
    
    for item in list_aggr_mean:
        group_mean = df[item].groupby(df['idhogar']).mean()
        new_col = item + '_aggr_MEAN'
        df_2[new_col] = group_mean
        

    for item in list_other:
        for function in ['mean','std','min','max','sum']:
            group = df[item].groupby(df['idhogar']).agg(function)
            new_col = item + '_' + function.upper()
            df_2[new_col] = group
            
            
    df_2 = df_2.reset_index()
    df = pd.merge(df, df_2, on = 'idhogar')
    df.fillna(value=0, inplace=True) #check why we're doing this !
    return df
   


In [358]:
#not working and clearly doesnt make sense to me ...
for elem in [train, test]:
    feature_creation_function(elem)
    preprocess(elem) #reencode idhogar
    clean_column(elem)
    recoding_ordinal(elem)
    elem = agg_function_household(elem)
    clean_column(elem)

In [174]:
#Apply to Train
preprocess(train)
feature_creation_function(train)
clean_column(train)
recoding_ordinal(train)
train = agg_function_household(train)


#Apply to test
preprocess(test)
feature_creation_function(test)
clean_column(test)
recoding_ordinal(test)
test = agg_function_household(test)


### Feature selection and model preparation

In [155]:
#Feature correlation with Target ranknig - We see that some feature we have re-ingeneered are working great ! 
train.corr()['Target'].map(lambda x: abs(x)).sort_values(ascending = False)

Target                       1.000000
escolari_MEAN                0.449353
escolari_MAX                 0.388633
fe_children_over_working     0.376389
meaneduc                     0.333932
house_qual                   0.329522
hogar_nin                    0.328199
r4t1                         0.316745
SQBhogar_nin                 0.311186
danger                       0.310006
instlevel8_aggr_MEAN         0.306825
walls                        0.306293
escolari_MIN                 0.305241
cielorazo                    0.304421
floor                        0.303165
escolari                     0.302305
fe_mobile_density            0.301448
SQBescolari                  0.296577
eviv3                        0.294222
epared3                      0.292451
fe_children_fraction         0.290035
fe_human_bed_density         0.289110
overcrowding                 0.289110
fe_person_per_room           0.282458
pisomoscer                   0.280284
instlevel1_aggr_MEAN         0.278413
SQBmeaned   

In [165]:
#To improve our variance estimator, we want to delete highly correlated feature - We choose a 0.95 threshold above wich we delete the feature
corr_matrix = train.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

to_drop = [column for column in upper.columns if any(abs(upper[column]) > 0.95)]
print(to_drop)

                            v2a1    hacdor     rooms    hacapo      v14a  \
v2a1                         NaN -0.025479  0.101634 -0.030867  0.021742   
hacdor                       NaN       NaN -0.233369  0.652594 -0.154152   
rooms                        NaN       NaN       NaN -0.213368  0.131587   
hacapo                       NaN       NaN       NaN       NaN -0.133241   
v14a                         NaN       NaN       NaN       NaN       NaN   
refrig                       NaN       NaN       NaN       NaN       NaN   
v18q                         NaN       NaN       NaN       NaN       NaN   
v18q1                        NaN       NaN       NaN       NaN       NaN   
r4h1                         NaN       NaN       NaN       NaN       NaN   
r4h2                         NaN       NaN       NaN       NaN       NaN   
r4h3                         NaN       NaN       NaN       NaN       NaN   
r4m1                         NaN       NaN       NaN       NaN       NaN   
r4m2        

In [176]:
def drop_var(df):
    df.drop(['tamhog', 'hhsize', 'coopele', 'female', 'hogar_total', 'area2', 'SQBage', 'SQBhogar_total', 'agesq', 'fe_human_bed_density', 'fe_person_per_room', 'power', 'male_aggr_MEAN', 'female_aggr_MEAN'],axis=1, inplace=True)
#Column wth more than 0.95% corr to any other column 
   
    df.drop(['elimbasu5','noelec', 'public', 'planpri', 'epared1', 'epared2', 'epared3', 'etecho1', 'etecho2', 'etecho3', 'eviv1', 'eviv2', 'eviv3'],axis=1, inplace=True)
#column we have re-ingeneered


    

In [177]:
#dropping variables
drop_var(train)
drop_var(test)

In [178]:
X = train.query('parentesco1==1')

# pull out the target variable
y = X['Target'] - 1 # this is done to bing input labels [1,2,3,4] in agreement with lightgbm [0,1,2,3]
X = X.drop(['Target'], axis=1)

### Model building - 3/4 models: Logistic regression - RF - LGBM - XGBoost

In [ ]:
#https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro

In [ ]:
#https://www.kaggle.com/skooch/xgboost